In [1]:
import numpy as np
import pandas as pd
from sklearn.utils.multiclass import unique_labels
import os

In [3]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 665.8 kB/s eta 0:00:000:00:010:00:01:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [6]:
pip install keras

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
from keras.models import Sequential
from keras.applications import DenseNet121, ResNet50,VGG16
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
from keras.datasets import cifar10

In [10]:
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

170498071/170498071 [==============================] - 52s 0us/step


In [11]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.3)


In [12]:
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((35000, 32, 32, 3), (35000, 1))
((15000, 32, 32, 3), (15000, 1))
((10000, 32, 32, 3), (10000, 1))


In [13]:
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)

In [14]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [15]:
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((35000, 32, 32, 3), (35000, 10))
((15000, 32, 32, 3), (15000, 10))
((10000, 32, 32, 3), (10000, 10))


In [17]:
#Data Augmentation
train_generator=ImageDataGenerator(rotation_range=2,horizontal_flip=True,
                                  zoom_range=.1)
val_generator=ImageDataGenerator(rotation_range=2,horizontal_flip=True,
                                  zoom_range=.1)
test_generator=ImageDataGenerator(rotation_range=2,horizontal_flip=True,
                                  zoom_range=.1)

In [18]:
#fit the augmentation method to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [19]:
y_train.shape

(35000, 10)

In [20]:
basemodel_densenet=DenseNet121(include_top=False,
                              weights='imagenet',
                               input_shape=(32,32,3),
                               classes=y_train.shape[1]
                              )
basemodel_resnet=ResNet50(weights='imagenet',
                         include_top=False,
                          input_shape=(32,32,3),
                          classes=y_train.shape[1]
                         )

basemodel_vgg=VGG16(weights='imagenet',
                         include_top=False,
                          input_shape=(32,32,3),
                          classes=y_train.shape[1]
                         )

#for layer in basemodel_desnet.layers:
#layer.trainable=False

#for layer in basemodel_resnet.layers:
#layer.trainable=False

#for layer in basemodel_vgg.layers:
#layer.trainable=False

2023-01-03 11:47:42.971412: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-03 11:47:42.971437: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-03 11:47:42.971455: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mkunwor): /proc/driver/nvidia/version does not exist
2023-01-03 11:47:42.972506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


58889256/58889256 [==============================] - 47s 1us/step


In [21]:
len(basemodel_densenet.layers)

427

In [22]:
model_1=Sequential()
model_1.add(basemodel_densenet)
model_1.add(Flatten())

In [23]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 1, 1, 1024)        7037504   
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
Total params: 7,037,504
Trainable params: 6,953,856
Non-trainable params: 83,648
_________________________________________________________________


In [24]:
#Add tge Sense layers along witg activation and batch normalization

model_1.add(Dense(64,activation=('relu'),input_dim=512))
model_1.add(Dense(32,activation=('relu')))
model_1.add(Dropout(.2)) #Adding a dropout layer that will randomly drop 30% of the weights
model_1.add(Dense(10,activation=('softmax')))#This is classification layer

In [25]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 1, 1, 1024)        7037504   
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 64)                65600     
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 7,105,514
Trainable params: 7,021,866
Non-

In [27]:
batch_size=100
epochs=10
learn_rate=.001

sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
adam=Adam(learning_rate=learn_rate,beta_1=0.9,beta_2=0.999,epsilon=None,decay=0.0,amsgrad=False)

In [28]:
model_1.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model_1.fit(train_generator.flow(x_train,y_train,batch_size),
           epochs=epochs,
           steps_per_epoch=x_train.shape[0]//batch_size,
           validation_data=val_generator.flow(x_val,y_val,batch_size=batch_size),
            verbose=1
           )

Epoch 1/10
350/350 [==============================] - 314s 875ms/step - loss: 1.6132 - accuracy: 0.4386 - val_loss: 1.0982 - val_accuracy: 0.6411
Epoch 2/10
350/350 [==============================] - 319s 910ms/step - loss: 0.9371 - accuracy: 0.6934 - val_loss: 0.7765 - val_accuracy: 0.7421
Epoch 3/10
350/350 [==============================] - 334s 954ms/step - loss: 0.7420 - accuracy: 0.7588 - val_loss: 0.7072 - val_accuracy: 0.7659
Epoch 4/10
350/350 [==============================] - 332s 948ms/step - loss: 0.6339 - accuracy: 0.7983 - val_loss: 0.6273 - val_accuracy: 0.7903
Epoch 5/10
350/350 [==============================] - 335s 957ms/step - loss: 0.5514 - accuracy: 0.8230 - val_loss: 0.6458 - val_accuracy: 0.7879
Epoch 6/10
310/350 [=========================>....] - ETA: 35s - loss: 0.4985 - accuracy: 0.8384

In [ ]:
f,ax=plt.subplot(2,1) #creates 2 subplots under 1 column

#Assign the first subplot to graph training loss and validation loss
ax[0].plot(model_1.history.history['loss'],color='b',label='Training loss')
ax[0].plot(model_1.history.history['loss'],color='r',label='Validation loss')
ax[0].legend()

#Next lets plot the training accuracy and validation accuracy
ax[1].plot(model_1.history.history['accuracy'],color='b',label='Training Accuracy')
ax[1].plot(model_1.history.history['val_accuracy'],color='b',label='Validation Accuracy')
ax[1].legend()

In [ ]:
#serialize model to json
model_json=model+1.to_json()
with open('model.json','w') as json.file:
    json_file.write(model_json)
#serialize weights to HDF5
model_1.save_weights('model.h5')
print("saved model to disk")

In [ ]:
model=load_model('model.h5')

In [ ]:
y_pred=model_1.predict_classes(x_test)
y_true=np.argmax(y_test,axis=1)

In [ ]:
report=classification_report(y_true,y_pred)


In [ ]:
print(report)

In [ ]:
confusion_matrix(y_true,y_pred)